In [6]:
# Dataset - https://www.kaggle.com/datasets/salader/dogs-vs-cats

In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

'kaggle' is not recognized as an internal or external command,
operable program or batch file.


Extraemos las imagenes del acrhivo zip

In [12]:
import zipfile
zip_ref = zipfile.ZipFile('archive.zip', 'r')
zip_ref.extractall('')
zip_ref.close()

# Split de datos

In [13]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Ruta a la carpeta PetImages que contiene las carpetas Cat y Dog
source_dir = "PetImages"

# Crear las carpetas Train y Test
os.mkdir(os.path.join(source_dir, "Train"))
os.mkdir(os.path.join(source_dir, "Test"))

# Crear las carpetas Cat y Dog dentro de Train y Test
os.mkdir(os.path.join(source_dir, "Train", "Cat"))
os.mkdir(os.path.join(source_dir, "Train", "Dog"))
os.mkdir(os.path.join(source_dir, "Test", "Cat"))
os.mkdir(os.path.join(source_dir, "Test", "Dog"))

# Dividir las imágenes de gatos en conjuntos de entrenamiento y prueba
cat_dir = os.path.join(source_dir, "Cat")
cat_filenames = os.listdir(cat_dir)
cat_train_filenames, cat_test_filenames = train_test_split(cat_filenames, test_size=0.2, random_state=42)

# Mover las imágenes de gatos a las carpetas correspondientes en Train y Test
for filename in cat_train_filenames:
    source = os.path.join(cat_dir, filename)
    destination = os.path.join(source_dir, "Train", "Cat", filename)
    shutil.move(source, destination)

for filename in cat_test_filenames:
    source = os.path.join(cat_dir, filename)
    destination = os.path.join(source_dir, "Test", "Cat", filename)
    shutil.move(source, destination)

# Dividir las imágenes de perros en conjuntos de entrenamiento y prueba
dog_dir = os.path.join(source_dir, "Dog")
dog_filenames = os.listdir(dog_dir)
dog_train_filenames, dog_test_filenames = train_test_split(dog_filenames, test_size=0.2, random_state=42)

# Mover las imágenes de perros a las carpetas correspondientes en Train y Test
for filename in dog_train_filenames:
    source = os.path.join(dog_dir, filename)
    destination = os.path.join(source_dir, "Train", "Dog", filename)
    shutil.move(source, destination)

for filename in dog_test_filenames:
    source = os.path.join(dog_dir, filename)
    destination = os.path.join(source_dir, "Test", "Dog", filename)
    shutil.move(source, destination)


In [15]:
import os

train_dir = "Train"
test_dir = "Test"

cat_train_dir = os.path.join('PetImages', train_dir, "Cat")
dog_train_dir = os.path.join('PetImages', train_dir, "Dog")
cat_test_dir = os.path.join('PetImages', test_dir, "Cat")
dog_test_dir = os.path.join('PetImages', test_dir, "Dog")

print("Cantidad de imágenes en la carpeta de entrenamiento de gatos:", len(os.listdir(cat_train_dir)))
print("Cantidad de imágenes en la carpeta de entrenamiento de perros:", len(os.listdir(dog_train_dir)))
print("Cantidad de imágenes en la carpeta de prueba de gatos:", len(os.listdir(cat_test_dir)))
print("Cantidad de imágenes en la carpeta de prueba de perros:", len(os.listdir(dog_test_dir)))


Cantidad de imágenes en la carpeta de entrenamiento de gatos: 10000
Cantidad de imágenes en la carpeta de entrenamiento de perros: 10000
Cantidad de imágenes en la carpeta de prueba de gatos: 2501
Cantidad de imágenes en la carpeta de prueba de perros: 2501


In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout

In [17]:
# generators
train_ds = keras.utils.image_dataset_from_directory(
    directory = 'PetImages/Train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

validation_ds = keras.utils.image_dataset_from_directory(
    directory = 'PetImages/Test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(256,256)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [18]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [19]:
# create CNN model

model = Sequential()

model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(256,256,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 batch_normalization (BatchN  (None, 254, 254, 32)     128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 batch_normalization_1 (Batc  (None, 125, 125, 64)     256       
 hNormalization)                                                 
                                                        

In [21]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [22]:
history = model.fit(train_ds,epochs=1,validation_data=validation_ds)

 86/625 [===>..........................] - ETA: 13:41 - loss: 4.0813 - accuracy: 0.5509

InvalidArgumentError: Graph execution error:

Input is empty.
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_2691]

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],color='red',label='train')
plt.plot(history.history['val_accuracy'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
# ways to reduce overfitting

# Add more data
# Data Augmentation -> next video
# L1/L2 Regularizer
# Dropout
# Batch Norm
# Reduce complexity

In [ ]:
import cv2

In [ ]:
test_img = cv2.imread('/content/cat.jpg')

In [ ]:
plt.imshow(test_img)

In [ ]:
test_img.shape

In [ ]:
test_img = cv2.resize(test_img,(256,256))

In [ ]:
test_input = test_img.reshape((1,256,256,3))

In [ ]:
model.predict(test_input)